<a href="https://colab.research.google.com/github/SimplifyHub19/ACED/blob/main/my_stock_screener_v0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import yfinance as yf
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def create_session():
    """Create a session with retries for handling connection issues."""
    session = requests.Session()
    retries = Retry(total=5, backoff_factor=0.3, status_forcelist=[500, 502, 503, 504])
    session.mount("https://", HTTPAdapter(max_retries=retries))
    session.headers.update({'User-Agent': 'Mozilla/5.0'})
    return session

def fetch_stock_data(ticker, session):
    """Fetch historical stock data from Yahoo Finance."""
    try:
        stock = yf.Ticker(ticker, session=session)
        hist = stock.history(period="6mo")
        return stock, hist
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None, None

def fundamental_filter(stock):
    """Apply fundamental screening criteria."""
    try:
        info = getattr(stock, 'info', None)
        if not info or not isinstance(info, dict):
            print("Error: No fundamental data available.")
            return False

        pe_ratio = info.get("trailingPE", None)
        pb_ratio = info.get("priceToBook", None)
        roe = info.get("returnOnEquity", None)
        market_cap = info.get("marketCap", None)

        if pe_ratio and pe_ratio < 20 and pb_ratio and pb_ratio < 3 and roe and roe > 0.1:
            return True  # Stock passes fundamental criteria
    except Exception as e:
        print(f"Error processing fundamental data: {e}")
    return False

def technical_filter(hist):
    """Apply technical screening criteria."""
    if hist is None or hist.empty:
        print("Skipping technical filter due to missing data.")
        return False
    try:
        hist["50_MA"] = hist["Close"].rolling(window=50).mean()
        hist["200_MA"] = hist["Close"].rolling(window=200).mean()
        rsi_period = 14
        delta = hist["Close"].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=rsi_period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=rsi_period).mean()
        rs = gain / loss
        hist["RSI"] = 100 - (100 / (1 + rs))

        latest_price = hist.iloc[-1]
        if latest_price["Close"] > latest_price["50_MA"] > latest_price["200_MA"] and latest_price["RSI"] < 70:
            return True  # Stock passes technical criteria
    except Exception as e:
        print(f"Error processing technical data: {e}")
    return False

def screen_stocks(tickers):
    """Screen multiple stocks based on fundamental and technical filters."""
    selected_stocks = []
    session = create_session()
    for ticker in tickers:
        print(f"Processing {ticker}...")
        stock, hist = fetch_stock_data(ticker, session)
        if stock and hist is not None and fundamental_filter(stock) and technical_filter(hist):
            selected_stocks.append(ticker)
    return selected_stocks

# Example: List of Malaysian stock tickers (replace with real tickers)
malaysia_tickers = ["5347.KL", "3182.KL", "1155.KL"]  # Sample tickers

filtered_stocks = screen_stocks(malaysia_tickers)
print("Selected stocks:", filtered_stocks)


Processing 5347.KL...
Processing 3182.KL...
Processing 1155.KL...
Selected stocks: []
